# Rule: **solve_sector_network**

**Outputs**

- results/postnetworks/`elec_s{simpl}_{clusters}_l{ll}_{opts}_{sector_opts}_{horizons}.nc`

In [ ]:
######################################## Parameters

### Run
name = '04_updated_ALL'
prefix = '1H'

### Network
simpl = ''
clusters = 35
ll = 'v1.0'
opts = ''
sector_opts = ''
horizons = '2030'

In [ ]:
##### Import packages
import pypsa
import pandas as pd
import cartopy.crs as ccrs
import geopandas as gpd
import matplotlib.pyplot as plt
import yaml
import os 
import sys


##### Import local functions
sys.path.append(os.path.abspath(os.path.join('..')))
import functions as xp


##### Read params.yaml
with open('../params.yaml', 'r') as configfile:
    params = yaml.safe_load(configfile)


##### Ignore warnings
import warnings
warnings.filterwarnings('ignore', category=UserWarning)


##### Region files
file_regions_onshore = f'regions_onshore_elec_s{simpl}_{clusters}.geojson'
file_regions_offshore = f'regions_offshore_elec_s{simpl}_{clusters}.geojson'
path_regions = f'{params['rootpath']}/resources/{prefix}/{name}/'
gdf_regions_onshore = gpd.read_file(path_regions+file_regions_onshore)
gdf_regions_offshore = gpd.read_file(path_regions+file_regions_offshore)


##### NUTS files (must contain at least columns 'NUTS_ID' and 'geometry')
file_NUTS2 = 'NUTS2_ES.geojson'
file_NUTS3 = 'NUTS3_ES.geojson'
path_NUTS = f'{params['rootpath']}/data_ES/nuts/'
gdf_NUTS2 = gpd.read_file(path_NUTS+file_NUTS2)
gdf_NUTS3 = gpd.read_file(path_NUTS+file_NUTS3)


## `elec_s{simpl}_{clusters}_l{ll}_{opts}_{sector_opts}_{horizons}.nc`

Load the network and show its components.

In [ ]:
file = f'elec_s{simpl}_{clusters}_l{ll}_{opts}_{sector_opts}_{horizons}.nc'
path = f'{params['rootpath']}/results/{prefix}/{name}/postnetworks/'

n = pypsa.Network(path+file)

n

Plot the network.

In [ ]:
#################### Parameters
line_widths = 1*n.lines.s_nom / 1e3
link_widths = 1*n.links.p_nom / 1e3



#################### Figure
fig_size = [12,12]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Add network
n.plot(ax=ax, line_widths=line_widths, link_widths=link_widths, bus_sizes=params['bus_sizes'], bus_colors=params['bus_colors'], boundaries=params['boundaries_offshore'])

### Add regions_onshore
xp.map_add_region(ax, gdf_regions_onshore, params['map_add_region'])

### Add regions_offshore
xp.map_add_region(ax, gdf_regions_offshore, params['map_add_region'], is_offshore=True)

### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Variable: `n.generators`

Optimal capacity has been included.

Place `n.generators` in a dataFrame and show its content.

In [ ]:
gg = n.generators

gg.head()

#### Summary

What is the aggregated capacity per carrier? 

And the optimal capacity?

In [ ]:
gg.groupby('carrier').agg(
    Total_capacity=pd.NamedAgg(column='p_nom', aggfunc='sum'),
    Optimal_capacity=pd.NamedAgg(column='p_nom_opt', aggfunc='sum'),
)

#### Maps

Plot a map showing a particular feature of a generation carrier at each region.

Then, show another map with the same information aggregated to a certain NUTS level (weighted with ovelap area).

In [ ]:
#################### Parameters

### Select carrier
carrier = 'offwind-float'

### Select feature (uncomment one of the following):
# feature = 'area' 
# feature = 'p_nom'
# feature = 'p_nom_density'
# feature = 'p_nom_max'
# feature = 'p_nom_max_density'
# feature = 'p_nom_max_ratio'
feature = 'p_nom_opt'
# feature = 'p_nom_opt_density'
# feature = 'p_nom_opt_max_ratio'



#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value



#################### Figure
fig_size = [12,6]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Define gdf_regions
if 'off' in carrier:
    gdf_regions = gdf_regions_offshore
    is_offshore = True
else:
    gdf_regions = gdf_regions_onshore
    is_offshore = False


### Add regions
xp.map_add_region(ax, gdf_regions, params['map_add_region'], is_offshore=is_offshore)

### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Add network feature at regions
xp.map_network_generators(carrier, n, feature, ax, gdf_regions, params['map_network_generators'], params_local)

In [ ]:
#################### Parameters

### Select NUTS level
NUTS_level = 3



#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value


if is_offshore:
    print('Aggregation at NUTS level for offshore is not possible')
else:
    #################### Figure
    fig_size = [12,6]
    crs = ccrs.PlateCarree()

    fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


    ### Define NUTS file
    if NUTS_level==2:
        gdf_NUTS = gdf_NUTS2
    if NUTS_level==3:
        gdf_NUTS = gdf_NUTS3    


    ### Add regions
    xp.map_add_region(ax, gdf_regions, params['map_add_region'], is_offshore=is_offshore)

    ### Add map features
    xp.map_add_features(ax, params['map_add_features'])

    ### Add network feature aggregated at NUTS regions
    xp.map_NUTS_generators(carrier, n, f'{feature}_NUTS', ax, gdf_regions, gdf_NUTS, params['map_NUTS_generators'], params_local)


del gdf_regions

### Variable: `n.global_constraints`

A constraint for positive co2 sequestered has been added.

Place `n.global_constraints` in a dataFrame and show its content.

In [ ]:
gc = n.global_constraints

gc

### Variable: `n.links`

Optimal capacity has been included.

Place `n.links` in a dataFrame and show its content.

In [ ]:
lk = n.links

lk.head()

#### Summary

Note that links represent different elements in the network. The aggregated capacity per carrier may need to be divided by the efficiency to properly reflect the capacity. For example, for CCGT, the link represents the conversion from gas to electricity, and the associated capacity refers to the input node. However, the CCGT capacity refers to the power capacity (output node). For this reason, no summary for links is here presented.

#### Maps

Plot a map showing a particular feature of a link carrier at each region.

Then, show another map with the same information aggregated to a certain NUTS level (weighted with ovelap area).

In [ ]:
#################### Parameters

### Select carrier (uncomment one of the following)
# carrier = 'CCGT'
# carrier = 'OCGT'
# carrier = 'H2 Electrolysis'
# carrier = 'H2 Fuel Cell'
# carrier = 'battery charger'
carrier = 'battery discharger'

### Select feature (uncomment one of the following):
# feature = 'area' 
# feature = 'p_nom_e'
feature = 'p_nom_e_opt'



#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value



#################### Figure
fig_size = [12,6]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Add regions
xp.map_add_region(ax, gdf_regions_onshore, params['map_add_region'], is_offshore=is_offshore)

### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Add network feature at regions
xp.map_network_links(carrier, n, feature, ax, gdf_regions_onshore, params['map_network_links'], params_local)

In [ ]:
#################### Parameters

### Select NUTS level
NUTS_level = 3



#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value



#################### Figure
fig_size = [12,6]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Define NUTS file
if NUTS_level==2:
    gdf_NUTS = gdf_NUTS2
if NUTS_level==3:
    gdf_NUTS = gdf_NUTS3    


### Add regions
xp.map_add_region(ax, gdf_regions_onshore, params['map_add_region'], is_offshore=is_offshore)

### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Add network feature aggregated at NUTS regions
xp.map_NUTS_links(carrier, n, f'{feature}_NUTS', ax, gdf_regions_onshore, gdf_NUTS, params['map_NUTS_links'], params_local)

### Variable: `n.stores`

Optimal capacity has been included.

Place `n.stores` in a dataFrame and show its content.

In [ ]:
st = n.stores

st.head()

#### Maps

Plot a map showing a particular feature of a store carrier at each region.

Then, show another map with the same information aggregated to a certain NUTS level (weighted with ovelap area).

In [ ]:
#################### Parameters

### Select carrier (uncomment one of the following):
carrier = 'battery'
# carrier = 'H2 Store'

### Select feature (uncomment one of the following):
feature = 'e_nom_opt'



#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value



#################### Figure
fig_size = [12,6]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Add regions
xp.map_add_region(ax, gdf_regions_onshore, params['map_add_region'], is_offshore=is_offshore)

### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Add network feature at regions
xp.map_network_stores(carrier, n, feature, ax, gdf_regions_onshore, params['map_network_stores'], params_local)

In [ ]:
#################### Parameters

### Select NUTS level
NUTS_level = 3



#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value



#################### Figure
fig_size = [12,6]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Define NUTS file
if NUTS_level==2:
    gdf_NUTS = gdf_NUTS2
if NUTS_level==3:
    gdf_NUTS = gdf_NUTS3    


### Add regions
xp.map_add_region(ax, gdf_regions_onshore, params['map_add_region'], is_offshore=is_offshore)

### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Add network feature aggregated at NUTS regions
xp.map_NUTS_stores(carrier, n, f'{feature}_NUTS', ax, gdf_regions_onshore, gdf_NUTS, params['map_NUTS_stores'], params_local)

### Analysis: Grid expansion

If enabled, optimation includes grid expansion for lines and links.

Where was grid expansion required?

Place `n.lines` and `n.links` in dataFrames.

In [ ]:
ln = n.lines
lk = n.links

Plot the network showing the increase of line capacities (if any).

In [ ]:
#################### Parameters
line_widths = 1*(n.lines.s_nom_opt-n.lines.s_nom) / 1e3
link_widths = 1*(n.links.p_nom_opt-n.links.p_nom) / 1e3



#################### Figure
fig_size = [12,12]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Add network
n.plot(ax=ax, line_widths=line_widths, link_widths=link_widths, bus_sizes=params['bus_sizes'], bus_colors=params['bus_colors'], boundaries=params['boundaries_offshore'])

### Add regions_onshore
xp.map_add_region(ax, gdf_regions_onshore, params['map_add_region'])

### Add regions_offshore
xp.map_add_region(ax, gdf_regions_offshore, params['map_add_region'], is_offshore=True)

### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Analysis: Grid congestion

Where are the bottlenecks in the grid from the optimal dispatch?

Place `n.lines[p0]` and `n.links[p0] in dataFrames.

In [ ]:
lit_p0 = n.lines_t['p0']
lkt_p0 = n.links_t['p0']

Plot the network showing the level of congestion.

In [ ]:
#################### Parameters

### Select criterion
cong_criterion = 'mean'
cong_criterion = 'Q95'

### Define line and link widths
line_widths = 1*(n.lines.s_nom_opt-n.lines.s_nom) / 1e3
link_widths = 1*(n.links.p_nom_opt-n.links.p_nom) / 1e3



#################### Expand n.lines and n.links to provide colour information


In [ ]:


#################### Figure
fig_size = [12,12]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Add network
n.plot(ax=ax, line_widths=line_widths, link_widths=link_widths, bus_sizes=params['bus_sizes'], bus_colors=params['bus_colors'], boundaries=params['boundaries_offshore'])

### Add regions_onshore
xp.map_add_region(ax, gdf_regions_onshore, params['map_add_region'])

### Add regions_offshore
xp.map_add_region(ax, gdf_regions_offshore, params['map_add_region'], is_offshore=True)

### Add map features
xp.map_add_features(ax, params['map_add_features'])